In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
data=pd.read_csv("Data_limpia.csv")

## Modelos

### Modelo con todas las variables

Variables explicativas

In [4]:
data.columns

Index(['Rented Bike Count', 'Hour', 'Temperature(C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day'],
      dtype='object')

In [12]:
# Definir las variables explicativas del modelo. Es decir todas menos la variable de respuesta
features = ['Hour', 'Temperature(C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day']

# Guardar el df con solo variables explicativas
X = data[features]

# Convertir las variable de tipo texto en categoricas.
# Se usa el dtype=int, para que lo convierta en 1-0 y no en true-false para facilidad del modelo
# Se usa drop_first=true, para eliminar la primera opción de las variables categoricas y evitar el problema de multicolinealidad
X = pd.get_dummies(X, columns=['Seasons', 'Holiday', 'Functioning Day'], dtype=int, drop_first=True)

X.head()

X.shape

(8743, 14)

Variable de Respuesta

In [13]:
# Definir la variable de respuesta

Y=data["Rented Bike Count"]
Y.head()

Y.shape

(8743,)

Regresión MCO

In [14]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1,test_size=0.2)

# agregar constante explíticamente
X_train = sm.add_constant(X_train)

In [15]:
# regresión usando mínimos cuadrados ordinarios (ordinary least squares - OLS) 
model = sm.OLS(y_train, X_train).fit()

# resumen de resultados
print(model.summary())

                             OLS Regression Results                            
Dep. Variable:       Rented Bike Count   R-squared:                       0.559
Model:                             OLS   Adj. R-squared:                  0.558
Method:                  Least Squares   F-statistic:                     632.2
Date:              dom., 08 sept. 2024   Prob (F-statistic):               0.00
Time:                         21:46:06   Log-Likelihood:                -52335.
No. Observations:                 6994   AIC:                         1.047e+05
Df Residuals:                     6979   BIC:                         1.048e+05
Df Model:                           14                                         
Covariance Type:             nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const         

Verificar multicolinealidad

In [20]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Se crea un df, en el que se añade una columna con constantes que representa la columna de unos en MCO
df_MCO = add_constant(X)

# Se crea un DF vacio para poder calcular el VIF de cada variable
vif_df = pd.DataFrame()

# Se crea una columna con el nombre de Variable Explicativa, a la que cada fila representa una variable explicativa
vif_df["Variable Explicativa"] = df_MCO.columns

# Se calcula el VIF de cada variable

vif_values = []
for i in range(df_MCO.shape[1]):
    vif = variance_inflation_factor(df_MCO.values,i)

    vif_values.append(vif)

vif_df["VIF"]=vif_values                                                 

# Filtra solo las filas correspondientes a las variables explicativas (no la constante)
vif_df = vif_df[vif_df["Variable Explicativa"] != "const"]

# Imprime los resultados
print(vif_df)

        Variable Explicativa         VIF
1                       Hour    1.214939
2             Temperature(C)  122.348253
3                Humidity(%)   29.903385
4           Wind speed (m/s)    1.302689
5           Visibility (10m)    1.721393
6   Dew point temperature(C)  165.512625
7    Solar Radiation (MJ/m2)    2.037762
8               Rainfall(mm)    1.099403
9              Snowfall (cm)    1.130734
10            Seasons_Spring    1.703757
11            Seasons_Summer    2.616806
12            Seasons_Winter    3.372716
13        Holiday_No Holiday    1.024113
14       Functioning Day_Yes    1.081420


Definir las variables explicativas

### Modelo Variables Significativas

In [21]:
# Definir las variables explicativas del modelo. Es decir todas menos la variable de respuesta
features = ['Hour', 'Humidity(%)',
       'Wind speed (m/s)', 'Dew point temperature(C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day']

# Guardar el df con solo variables explicativas
X = data[features]

# Convertir las variable de tipo texto en categoricas.
# Se usa el dtype=int, para que lo convierta en 1-0 y no en true-false para facilidad del modelo
# Se usa drop_first=true, para eliminar la primera opción de las variables categoricas y evitar el problema de multicolinealidad
X = pd.get_dummies(X, columns=['Seasons', 'Holiday', 'Functioning Day'], dtype=int, drop_first=True)

X.head()

X.shape

(8743, 12)

In [22]:
Y=data["Rented Bike Count"]
Y.head()

Y.shape

(8743,)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1,test_size=0.2)

# agregar constante explíticamente
X_train = sm.add_constant(X_train)

In [24]:
# regresión usando mínimos cuadrados ordinarios (ordinary least squares - OLS) 
model = sm.OLS(y_train, X_train).fit()

# resumen de resultados
print(model.summary())

                             OLS Regression Results                            
Dep. Variable:       Rented Bike Count   R-squared:                       0.559
Model:                             OLS   Adj. R-squared:                  0.558
Method:                  Least Squares   F-statistic:                     737.2
Date:              dom., 08 sept. 2024   Prob (F-statistic):               0.00
Time:                         22:08:03   Log-Likelihood:                -52337.
No. Observations:                 6994   AIC:                         1.047e+05
Df Residuals:                     6981   BIC:                         1.048e+05
Df Model:                           12                                         
Covariance Type:             nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const         

Multicolinealidad

In [25]:
# Se crea un df, en el que se añade una columna con constantes que representa la columna de unos en MCO
df_MCO = add_constant(X)

# Se crea un DF vacio para poder calcular el VIF de cada variable
vif_df = pd.DataFrame()

# Se crea una columna con el nombre de Variable Explicativa, a la que cada fila representa una variable explicativa
vif_df["Variable Explicativa"] = df_MCO.columns

# Se calcula el VIF de cada variable

vif_values = []
for i in range(df_MCO.shape[1]):
    vif = variance_inflation_factor(df_MCO.values,i)

    vif_values.append(vif)

vif_df["VIF"]=vif_values                                                 

# Filtra solo las filas correspondientes a las variables explicativas (no la constante)
vif_df = vif_df[vif_df["Variable Explicativa"] != "const"]

# Imprime los resultados
print(vif_df)

        Variable Explicativa       VIF
1                       Hour  1.204780
2                Humidity(%)  3.348454
3           Wind speed (m/s)  1.292186
4   Dew point temperature(C)  6.839555
5    Solar Radiation (MJ/m2)  1.818626
6               Rainfall(mm)  1.072968
7              Snowfall (cm)  1.120629
8             Seasons_Spring  1.619095
9             Seasons_Summer  2.610781
10            Seasons_Winter  3.229909
11        Holiday_No Holiday  1.022190
12       Functioning Day_Yes  1.080684


### Resumen de los coeficientes y R2

In [26]:
# Imprimir la tabla de coeficientes
coeficientes= model.params
print(coeficientes)

const                       416.428738
Hour                         26.442815
Humidity(%)                 -16.634052
Wind speed (m/s)             14.309891
Dew point temperature(C)     30.648292
Solar Radiation (MJ/m2)     -84.593649
Rainfall(mm)                -51.059521
Snowfall (cm)                45.481399
Seasons_Spring             -123.636163
Seasons_Summer             -188.796500
Seasons_Winter             -350.215048
Holiday_No Holiday          124.487162
Functioning Day_Yes         940.153683
dtype: float64


In [28]:
coeficientes.to_csv("coeficiente_df.csv",index=False,encoding="utf-8")

In [29]:
print(model.rsquared, model.rsquared_adj)

0.5589342251731537 0.5581760545245471
